In [ ]:
import ast
import os
import re
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm

plt.style.use("../project/nbody6/plot/style.mplstyle")

In [ ]:
load_dotenv()

OUTPUT_BASE = Path(os.getenv("OUTPUT_BASE"))
incl_stats_root_path = (OUTPUT_BASE / "inclination_stats").resolve()
if not incl_stats_root_path.is_dir():
    raise NotADirectoryError(f"{incl_stats_root_path} is NOT a directory")

full_incl_stats_df = pd.concat(
    [
        pd.read_csv(incl_stats_root_path / f)
        for f in os.listdir(incl_stats_root_path)
        if f.endswith(".csv")
    ],
    ignore_index=True,
)

In [ ]:
# parse the list-like strings into actual lists
def safe_eval(x):
    if pd.isna(x) or x in ("", "None", "nan", "NaN"):
        return None
    try:
        cleaned = re.sub(r"\bnan\b", "None", x)
        val = ast.literal_eval(cleaned)
    except (ValueError, SyntaxError, NameError, TypeError):
        return None
    if isinstance(val, list):
        return [np.nan if v is None else v for v in val]
    return val


for key in ["dist_pc", "dist_r_tidal", "radian", "degree"]:
    full_incl_stats_df[key] = full_incl_stats_df[key].apply(safe_eval)

In [ ]:
# overall statistics
total_n_wide_bin_sys = full_incl_stats_df["n_wide_bin_sys"].sum()
total_n_def_wide_bin_sys = full_incl_stats_df["n_defined_wide_bin_sys"].sum()
print(
    total_n_wide_bin_sys,
    total_n_def_wide_bin_sys,
    total_n_wide_bin_sys - total_n_def_wide_bin_sys,
    total_n_def_wide_bin_sys / total_n_wide_bin_sys,
)

In [ ]:
# misc config for calculation and plot
attr_keys = [
    "init_pos",
    "init_mass_lv",
    "init_metallicity",
    "init_gc_radius",
]

uni_timestamp_grid = np.arange(0, full_incl_stats_df["time"].max() + 1, 1)

mass_levels = list(range(1, 9))
mass_values = [12800 / (2**m) for m in mass_levels]
mass_map = dict(zip(mass_levels, mass_values))
cmap = mpl.colors.ListedColormap(
    [
        "#711415",
        "#ae311e",
        "#f37324",
        "#f6a020",
        "#f8cc1b",
        "#b5be2f",
        "#72b043",
        "#007f4e",
    ]
)
norm = mpl.colors.BoundaryNorm(boundaries=np.arange(0.5, 8.5 + 1, 1), ncolors=cmap.N)

attr_pairs = [(2, 4), (2, 8), (6, 8), (14, 4), (14, 8), (14, 12)]

fig_export_path = OUTPUT_BASE / "figures" / "binary_inclination"
fig_export_path.mkdir(parents=True, exist_ok=True)

## Mean $i$ Over Time

In [ ]:
exploded_incl_df = full_incl_stats_df.explode(
    ["degree"],
    ignore_index=True,
)[attr_keys + ["time", "degree", "n_defined_wide_bin_sys"]]
excloded_incl_df = exploded_incl_df.dropna(subset=["degree"]).reset_index(drop=True)

In [ ]:
# align distance statistics to the common timestamp grid
aligned_incl_df = pd.concat(
    [
        (
            g.set_index("time")["degree_mean"]
            .reindex(np.union1d(g["time"].to_numpy(), uni_timestamp_grid))
            .sort_index()
            .interpolate("index", limit_area="inside")
            .reindex(uni_timestamp_grid)
            .rename("degree_mean")
            .reset_index()
            .rename(columns={"index": "time"})
            .assign(**dict(zip(attr_keys, group_vals)))
        )
        for group_vals, g in tqdm(
            full_incl_stats_df[attr_keys + ["time", "degree_mean"]].groupby(
                attr_keys, observed=True, sort=False
            ),
            desc="Aligning",
            leave=True,
        )
    ],
    ignore_index=True,
).dropna(subset=["degree_mean"])

agg_incl_df = (
    aligned_incl_df.groupby(
        [
            "init_mass_lv",
            "init_metallicity",
            "init_gc_radius",
            "time",
        ],
        observed=True,
    )["degree_mean"]
    .agg(
        mean_incl_deg_mean="mean",
        median_incl_deg_mean="median",
        std_incl_deg_mean="std",
    )
    .reset_index()
)

In [ ]:
print(aligned_incl_df.head(), agg_incl_df.head(), sep="\n\n")

In [ ]:
for x_scale in ["linear", "symlog"]:
    for mass_lv in (
        mass_pbar := tqdm(
            mass_levels,
            total=len(mass_levels),
            leave=False,
            dynamic_ncols=True,
        )
    ):
        mass_pbar.set_description(f"Visualizing M{mass_lv}")

        group_df = exploded_incl_df[exploded_incl_df["init_mass_lv"] == mass_lv]
        if group_df.empty:
            continue

        fig, axs = plt.subplots(
            nrows=2,
            ncols=3,
            figsize=(19, 8),
            dpi=300,
            constrained_layout=True,
            gridspec_kw=dict(hspace=0.06, wspace=0.08),
        )

        for (init_metallicity, init_gc_radius), ax in zip(attr_pairs, axs.flat):
            mean_dist_df = agg_incl_df[
                (agg_incl_df["init_mass_lv"] == mass_lv)
                & (agg_incl_df["init_metallicity"] == init_metallicity)
                & (agg_incl_df["init_gc_radius"] == init_gc_radius)
            ]

            ax.plot(
                mean_dist_df["time"],
                mean_dist_df["mean_incl_deg_mean"],
                "-",
                # color="black",
                color=cmap(norm(mass_lv)),
                lw=1,
                alpha=0.8,
                label="mean",
                zorder=-1,
            )

            ax.plot(
                mean_dist_df["time"],
                mean_dist_df["median_incl_deg_mean"],
                "--",
                color=cmap(norm(mass_lv)),
                lw=1,
                alpha=0.8,
                label="median",
                zorder=-2,
            )

            ax.fill_between(
                mean_dist_df["time"],
                mean_dist_df["mean_incl_deg_mean"] - mean_dist_df["std_incl_deg_mean"],
                mean_dist_df["mean_incl_deg_mean"] + mean_dist_df["std_incl_deg_mean"],
                color=cmap(norm(mass_lv)),
                alpha=0.25,
                lw=0,
                label="std.",
                zorder=-3,
            )

            subgroup_df = group_df[
                (group_df["init_metallicity"] == init_metallicity)
                & (group_df["init_gc_radius"] == init_gc_radius)
                & (group_df["n_defined_wide_bin_sys"] > 0)
            ]

            ax.plot(
                subgroup_df["time"],
                subgroup_df["degree"],
                ".",
                # color=cmap(norm(mass_lv)),
                color="darkgrey",
                ms=1,
                alpha=0.2,
                # label=f"M={mass_lv}",
                zorder=-5,
            )

            if x_scale == "symlog":
                ax.set_xscale("symlog")
                ax.set_xlim(0, 400)
                ax.xaxis.set_major_locator(
                    mpl.ticker.SymmetricalLogLocator(
                        base=10.0,
                        linthresh=1,
                        subs=[1.0],
                    )
                )
                ax.xaxis.set_minor_locator(
                    mpl.ticker.FixedLocator(
                        [0] + [i * 10**j for j in range(0, 4) for i in range(2, 10)]
                    )
                )
            else:
                ax.set_xlim(0, 320)
                # ax.axvline(0, ls="--", lw=1, c="black", alpha=0.7)
                ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
                ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(20))
            ax.set_xlabel(r"$t\;[\mathrm{Myr}]$")

            ax.set_ylim(0, 180)
            ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(60, offset=30))
            ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(30))
            ax.set_ylabel(r"$i$ [$^\circ$]")

            ax.set_title(
                rf"$M_\mathrm{{init.}}={int(mass_map[mass_lv])}\,M_\odot,$"
                "\n"
                rf"$Z_{{init.}}={init_metallicity * 10e-4},\;"
                rf"R_\mathrm{{gc,\,init.}}={init_gc_radius}\ \mathrm{{kpc}}$",
                fontsize=20,
                y=1.02,
            )
            ax.grid(ls=":", lw=0.8, c="darkgrey")
            ax.legend(
                loc="upper center",
                ncol=3,
                prop={"style": "italic"},
                labelspacing=0.25,
                markerscale=0.5,
            )

        if mass_lv not in [1, 4, 8]:
            plt.close(fig)

        fig.savefig(
            fig_export_path / f"incl-{mass_lv}-{x_scale}.png",
            dpi=300,
            bbox_inches="tight",
        )

## $r_\mathrm{tidal}$-Normalized Mean Distance Over Time

In [ ]:
exploded_dist_r_tidal_df = full_incl_stats_df.explode(
    "dist_r_tidal",
    ignore_index=True,
)[attr_keys + ["time", "dist_r_tidal", "n_wide_bin_sys"]]
exploded_dist_r_tidal_df = exploded_dist_r_tidal_df[
    exploded_dist_r_tidal_df["n_wide_bin_sys"] > 0
]
exploded_dist_r_tidal_df["dist_r_tidal_mean"] = exploded_dist_r_tidal_df[
    "dist_r_tidal"
].apply(np.nanmean)


In [ ]:
# align distance statistics to the common timestamp grid
collapsed_dist_r_tidal_df = (
    exploded_dist_r_tidal_df.groupby(attr_keys + ["time"], observed=True)[
        "dist_r_tidal_mean"
    ]
    .mean()
    .reset_index()
)
aligned_dist_df = pd.concat(
    [
        (
            g.set_index("time")["dist_r_tidal_mean"]
            .reindex(np.union1d(g["time"].to_numpy(), uni_timestamp_grid))
            .sort_index()
            .interpolate("index", limit_area="inside")
            .reindex(uni_timestamp_grid)
            .rename("dist_r_tidal_mean")
            .reset_index()
            .rename(columns={"index": "time"})
            .assign(**dict(zip(attr_keys, group_vals)))
        )
        for group_vals, g in tqdm(
            collapsed_dist_r_tidal_df.groupby(attr_keys, observed=True, sort=False),
            desc="Aligning",
            leave=False,
        )
    ],
    ignore_index=True,
).dropna(subset=["dist_r_tidal_mean"])

agg_dist_df = (
    aligned_dist_df.groupby(
        [
            "init_mass_lv",
            "init_metallicity",
            "init_gc_radius",
            "time",
        ],
        observed=True,
    )["dist_r_tidal_mean"]
    .agg(
        mean_dist_r_tidal_mean="mean",
        median_dist_r_tidal_mean="median",
        std_dist_r_tidal_mean="std",
    )
    .reset_index()
)


In [ ]:
print(aligned_dist_df.head(), agg_dist_df.head(), sep="\n\n")

In [ ]:
for x_scale in ["linear", "symlog"]:
    for mass_lv in (
        mass_pbar := tqdm(
            mass_levels,
            total=len(mass_levels),
            leave=False,
            dynamic_ncols=True,
        )
    ):
        mass_pbar.set_description(f"Visualizing M{mass_lv}")

        group_df = exploded_dist_r_tidal_df[
            exploded_dist_r_tidal_df["init_mass_lv"] == mass_lv
        ]
        if group_df.empty:
            continue

        fig, axs = plt.subplots(
            nrows=2,
            ncols=3,
            figsize=(19, 8),
            dpi=300,
            constrained_layout=True,
            gridspec_kw=dict(hspace=0.06, wspace=0.08),
        )

        for (init_metallicity, init_gc_radius), ax in zip(attr_pairs, axs.flat):
            mean_dist_df = agg_dist_df[
                (agg_dist_df["init_mass_lv"] == mass_lv)
                & (agg_dist_df["init_metallicity"] == init_metallicity)
                & (agg_dist_df["init_gc_radius"] == init_gc_radius)
            ]

            ax.plot(
                mean_dist_df["time"],
                mean_dist_df["mean_dist_r_tidal_mean"],
                "-",
                # color="black",
                color=cmap(norm(mass_lv)),
                lw=1,
                alpha=0.8,
                label="mean",
                zorder=-1,
            )

            ax.plot(
                mean_dist_df["time"],
                mean_dist_df["median_dist_r_tidal_mean"],
                "--",
                # color="black",
                color=cmap(norm(mass_lv)),
                lw=1,
                alpha=0.8,
                label="median",
                zorder=-1,
            )

            ax.fill_between(
                mean_dist_df["time"],
                mean_dist_df["mean_dist_r_tidal_mean"]
                - mean_dist_df["std_dist_r_tidal_mean"],
                mean_dist_df["mean_dist_r_tidal_mean"]
                + mean_dist_df["std_dist_r_tidal_mean"],
                color=cmap(norm(mass_lv)),
                alpha=0.25,
                lw=0,
                label="std.",
                zorder=-3,
            )

            subgroup_df = group_df[
                (group_df["init_metallicity"] == init_metallicity)
                & (group_df["init_gc_radius"] == init_gc_radius)
                & (group_df["n_wide_bin_sys"] > 0)
            ]

            ax.plot(
                subgroup_df["time"],
                subgroup_df["dist_r_tidal_mean"],
                ".",
                # color=cmap(norm(mass_lv)),
                color="darkgrey",
                ms=1,
                alpha=0.1,
                # label=f"M={mass_lv}",
                zorder=-5,
            )

            if x_scale == "symlog":
                ax.set_xscale("symlog")
                ax.set_xlim(0, 400)
                ax.xaxis.set_major_locator(
                    mpl.ticker.SymmetricalLogLocator(
                        base=10.0,
                        linthresh=1,
                        subs=[1.0],
                    )
                )
                ax.xaxis.set_minor_locator(
                    mpl.ticker.FixedLocator(
                        [0] + [i * 10**j for j in range(0, 4) for i in range(2, 10)]
                    )
                )
            else:
                ax.set_xlim(-10, 320)
                ax.axvline(0, ls="--", lw=1, c="black", alpha=0.7)
                ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
                ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(20))
            ax.set_xlabel(r"$t\;[\mathrm{Myr}]$")

            ax.set_ylim(0, 1.4)
            ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.3, offset=0.1))
            ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.1))
            ax.set_ylabel(r"$d_\mathrm{dc}\;/\;r_\mathrm{tidal}$")

            ax.set_title(
                rf"$M_\mathrm{{init.}}={int(mass_map[mass_lv])}\,M_\odot,$"
                "\n"
                rf"$Z_{{init.}}={init_metallicity * 10e-4},\;"
                rf"R_\mathrm{{gc,\,init.}}={init_gc_radius}\ \mathrm{{kpc}}$",
                fontsize=20,
                y=1.02,
            )
            ax.grid(ls=":", lw=0.8, c="darkgrey")

            # set italic
            ax.legend(
                loc="upper center", ncol=3, prop={"style": "italic"}, labelspacing=0.25
            )

        if mass_lv not in [1, 4, 8]:
            plt.close(fig)

        fig.savefig(
            fig_export_path / f"dist_r_tidal-{mass_lv}-{x_scale}.png",
            dpi=300,
            bbox_inches="tight",
        )

In [ ]:
for x_scale in ["linear", "symlog"]:
    fig, axs = plt.subplots(
        nrows=2,
        ncols=3,
        figsize=(19, 8),
        dpi=300,
        constrained_layout=True,
        gridspec_kw=dict(hspace=0.06, wspace=0.08),
    )

    for mass_lv in (
        mass_pbar := tqdm(
            sorted(mass_levels, reverse=True),
            total=len(mass_levels),
            leave=False,
            dynamic_ncols=True,
        )
    ):
        mass_pbar.set_description(f"Plotting for mass level {mass_lv}")

        group_df = exploded_dist_r_tidal_df[
            exploded_dist_r_tidal_df["init_mass_lv"] == mass_lv
        ]
        if group_df.empty:
            continue

        for (init_metallicity, init_gc_radius), ax in zip(attr_pairs, axs.flat):
            mean_dist_df = agg_dist_df[
                (agg_dist_df["init_mass_lv"] == mass_lv)
                & (agg_dist_df["init_metallicity"] == init_metallicity)
                & (agg_dist_df["init_gc_radius"] == init_gc_radius)
            ]

            ax.plot(
                mean_dist_df["time"],
                mean_dist_df["mean_dist_r_tidal_mean"],
                "-",
                # color="black",
                color=cmap(norm(mass_lv)),
                lw=1,
                alpha=0.8,
                # label="Mean trend",
                zorder=-1,
            )

            # ax.fill_between(
            #     mean_dist_df["time"],
            #     mean_dist_df["mean_dist_r_tidal_mean"]
            #     - mean_dist_df["std_dist_r_tidal_mean"],
            #     mean_dist_df["mean_dist_r_tidal_mean"]
            #     + mean_dist_df["std_dist_r_tidal_mean"],
            #     color=cmap(norm(mass_lv)),
            #     alpha=0.25,
            #     lw=0,
            #     # label="Std. dev.",
            #     zorder=-3,
            # )

            # subgroup_df = group_df[
            #     (group_df["init_metallicity"] == init_metallicity)
            #     & (group_df["init_gc_radius"] == init_gc_radius)
            #     & (group_df["n_wide_bin_sys"] > 0)
            # ]

            # ax.plot(
            #     subgroup_df["time"],
            #     subgroup_df["dist_r_tidal_mean"],
            #     ".",
            #     # color=cmap(norm(mass_lv)),
            #     color="darkgrey",
            #     ms=1,
            #     alpha=0.1,
            #     label=f"M={mass_lv}",
            #     zorder=-5,
            # )

            if x_scale == "symlog":
                ax.set_xscale("symlog")
                ax.set_xlim(0, 400)
                ax.xaxis.set_major_locator(
                    mpl.ticker.SymmetricalLogLocator(
                        base=10.0,
                        linthresh=1,
                        subs=[1.0],
                    )
                )
                ax.xaxis.set_minor_locator(
                    mpl.ticker.FixedLocator(
                        [0] + [i * 10**j for j in range(0, 4) for i in range(2, 10)]
                    )
                )
            else:
                ax.set_xlim(-10, 320)
                ax.axvline(0, ls="--", lw=1, c="black", alpha=0.7)
                ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
                ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(20))
            ax.set_xlabel(r"$t\;[\mathrm{Myr}]$")

            ax.set_ylim(0, 1.2)
            ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.4, offset=0.2))
            ax.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.2))
            ax.set_ylabel(r"$d_\mathrm{dc}\;/\;r_\mathrm{tidal}$")

            ax.set_title(
                rf"$Z_{{init.}}={init_metallicity * 10e-4},\;"
                rf"R_\mathrm{{gc,\,init.}}={init_gc_radius}\ \mathrm{{kpc}}$",
                fontsize=20,
                y=1.02,
            )
            ax.grid(ls=":", lw=0.8, c="darkgrey")

    mass_values = [12800 / (2**m) for m in mass_levels]
    sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    cbar = fig.colorbar(
        sm,
        ax=axs,
        orientation="vertical",
        fraction=0.08,
        pad=0.03,
        # ticks=mass_levels,
    )
    cbar.ax.invert_yaxis()
    cbar.set_ticks(mass_levels)
    cbar.set_ticklabels([f"{int(mass)}" for mass in mass_values])
    cbar.set_label(
        # r"$M_\mathrm{tot.\,init.}=\dfrac{12\,800}{2^N}M_\odot$", fontsize=20
        r"$M_\mathrm{tot.\,init.} [M_\odot]$",
        fontsize=20,
    )
    cbar.ax.tick_params(which="major", direction="in", length=6)
    cbar.ax.tick_params(which="minor", length=0)

    fig.savefig(
        fig_export_path / f"dist_r_tidal-all-{x_scale}.png",
        dpi=300,
        bbox_inches="tight",
    )